In [55]:
import googlemaps
from datetime import datetime
from bs4 import BeautifulSoup




def get_key(api_key_path):
    """提供key檔案路徑，自動讀取內容並回傳"""
    with open(api_key_path, "r", encoding="UTF-8") as f:
        api_key = f.read()
    
    return api_key


def trans_unix_to_date(timestamp):
    """將UNIX電腦時間轉換為西元年月日"""
    # 將時間戳轉成 datetime 物件
    dt = datetime.fromtimestamp(timestamp)

    # 格式化成 "年-月-日"
    date_str = dt.strftime('%Y-%m-%d')

    return date_str

def if_exist(value):
    if value:
        return value
    else:
        return None


def get_place_id(api_key: str, key_word: str):
    """提供關鍵字（最好是店名+地址），透過gmap搜尋取得place_id"""
    gmaps = googlemaps.Client(key=api_key)
    search_result = gmaps.places(query=key_word, language='zh-TW')
    if len(search_result["results"]) != 0:
        place_id = search_result.get("results", {})[0]["place_id"]
        return place_id
    else:
        return None



def newest_review_date(review_list: list):
    """提供gmaps回傳的評論列表，回傳列表中最新的評論時間"""
    for i in review_list:
        time = 0
        if i["time"] > 0:
            time = i["time"]
    date = trans_unix_to_date(time)

    return date


def gmap_info(ori_name, api_key, place_id):
    """提供place_id，回傳名稱、營業狀態、營業時間、gmap評分、經緯度、gmap網址、最新評論日期"""
    if place_id != None:
        gmaps = googlemaps.Client(key=api_key)
        detail = gmaps.place(place_id=place_id, language='zh-TW')
        name = if_exist(detail["result"]['name'])
        business_status = if_exist(detail["result"]["business_status"])

        if detail["result"]["formatted_address"]:
            address = detail["result"]["formatted_address"]
        elif detail['result']['adr_address']:
            address = BeautifulSoup(detail['result']['adr_address'], "html.parser").text
        else:
            address = None
        
        phone = if_exist(detail["result"]["formatted_phone_number"].replace(" ", ""))
        opening_hours = if_exist(detail["result"]["opening_hours"]["weekday_text"])
        rating = if_exist(detail["result"]['rating'])
        rating_total = if_exist(detail["result"]["user_ratings_total"])
        geocode = if_exist(detail["result"]['geometry']['location'])
        map_url = if_exist(detail["result"]["url"])
        review_list = if_exist(detail["result"]['reviews'])
        
        if review_list != None:
            newest_review = newest_review_date(review_list)
        else:
            None

        place_info = {
            "name":name,
            "place_id":place_id,
            "business_status":business_status,
            "address":address,
            "phone":phone,
            "opening_hours":opening_hours,
            "rating":rating,
            "rating_total":rating_total,
            "geocode":geocode,
            "map_url":map_url,
            "newest_review":newest_review
        }
    else:
        place_info = {
            "name":ori_name,
            "place_id":None,
            "business_status":None,
            "address":None,
            "phone":None,
            "opening_hours":None,
            "rating":None,
            "rating_total":None,
            "geocode":None,
            "map_url":None,
            "newest_review":None
        }

    return place_info


def get_place_dict(api_key_path, name: str, address: str):
    key_word = name + " " + address

    api_key = get_key(api_key_path=api_key_path)
    place_id = get_place_id(api_key=api_key, key_word=key_word)
    place_dict = gmap_info(ori_name=name, api_key=api_key, place_id=place_id)

    return place_dict

In [37]:
api_key_path = r"C:\Users\add41\Documents\Data_Engineer\Project\TJR103-Group-1-Project\code\1_bevis\tool\map_api_key.txt"
key_word = "erg%*_#%5g3sd40"

api_key = get_key(api_key_path=api_key_path)
# place_id = get_place_id(api_key=api_key, key_word=key_word)
# gmap_info(api_key=api_key, place_id=place_id)

gmaps = googlemaps.Client(key=api_key)
search_result = gmaps.places(query=key_word, language='zh-TW')
# place_id = search_result.get("results", {})[0]["place_id"]

search_result["results"]


# gmaps = googlemaps.Client(key=api_key)
# detail = gmaps.place(place_id=place_id, language='zh-TW')

[]

In [42]:
import pandas as pd

data_path = r"C:\Users\add41\Documents\Data_Engineer\Project\example_data\petboo_fnb_retest.csv"
df = pd.read_csv(data_path)

df_sample = df.iloc[1:6]

df_sample

,city,district,name,address,phone,rating,rating_count,pet_rules,url
1,台北市,中正區,Grow Life 靠過來桌遊咖啡廳,100台灣台北市中正區新生南路一段160巷18-1號,0905 428 741,4.8,569.0,可落地、寵物友善,https://taiwan.petboo.co/place/2102
2,台北市,中正區,哈燒小食堂,100台灣台北市中正區寧波西街173-2號,02 2304 7720,4.6,159.0,可落地、寵物友善,https://taiwan.petboo.co/place/2061
3,台北市,中正區,花漾薇漫餐廳,100台灣台北市中正區羅斯福路三段244巷10弄19號,02 2365 6188,4.6,417.0,寵物友善,https://taiwan.petboo.co/place/2005
4,台北市,中正區,別處咖啡 Away cafe,10089台灣台北市中正區羅斯福路三段244巷9弄2-1號,02 2368 9713,4.5,162.0,可落地、寵物友善,https://taiwan.petboo.co/place/1857
5,台北市,中正區,uPoke 夏威夷生魚飯-水源門市（自來水博物館對面）,100台灣台北市中正區思源街16號,02 2368 8756,5.0,154.0,可落地、寵物友善,https://taiwan.petboo.co/place/1856


In [56]:
api_key_path = r"C:\Users\add41\Documents\Data_Engineer\Project\TJR103-Group-1-Project\code\1_bevis\tool\map_api_key.txt"

data_list = []
for index, row in df_sample.iterrows():
    name = row["name"]
    address = row["address"]
    # key_word = name + " " + address

    # api_key = get_key(api_key_path)
    # place_id = get_place_id(api_key=api_key, key_word=key_word)

    # place_dict = gmap_info(ori_name=name, api_key=api_key, place_id=place_id)
    # data_list.append(place_dict)
    data_list.append(get_place_dict(api_key_path, name, address))

In [57]:
df_final = pd.DataFrame(data_list)
df_final

,name,place_id,business_status,address,phone,opening_hours,rating,rating_total,geocode,map_url,newest_review
0,靠過來桌遊咖啡廳,ChIJ33MxVX6pQjQRRD957u4F5IU,OPERATIONAL,100台灣台北市中正區新生南路一段160巷18-1號,0905428741,"[星期一: 13:00 – 22:00, 星期二: 休息, 星期三: 13:00 – 22:...",4.9,1085,"{'lat': 25.0345369, 'lng': 121.5307933}",https://maps.google.com/?cid=9647842825495723844,2025-02-02
1,哈燒小食堂,ChIJo0lm5rypQjQRYa4S628K5XQ,OPERATIONAL,100台灣台北市中正區寧波西街173-2號,0223047720,"[星期一: 11:00 – 14:00, 17:00 – 20:00, 星期二: 11:00...",4.6,205,"{'lat': 25.0267852, 'lng': 121.5107325}",https://maps.google.com/?cid=8423150153867898465,2024-11-15
2,花漾薇漫餐廳,ChIJuUnPP0KpQjQRguNrS_dGJ7c,OPERATIONAL,100台灣台北市中正區羅斯福路三段244巷10弄19號,0223656188,"[星期一: 休息, 星期二: 11:00 – 20:30, 星期三: 11:00 – 20:...",4.5,697,"{'lat': 25.016115, 'lng': 121.531526}",https://maps.google.com/?cid=13197595260968756098,2025-06-10
3,別處咖啡 Away cafe,ChIJr52_JZ2pQjQRQ_UeWocNMTE,OPERATIONAL,10089台灣台北市中正區羅斯福路三段244巷9弄2-1號,0223689713,"[星期一: 13:00 – 21:00, 星期二: 13:00 – 21:00, 星期三: ...",4.5,486,"{'lat': 25.0169359, 'lng': 121.530442}",https://maps.google.com/?cid=3544629256701015363,2025-06-10
4,uMeal 優善糧｜食物樂高領導品牌｜台大水源店,ChIJhQuh3l2pQjQRUqi9n8z_NJ4,OPERATIONAL,100台灣台北市中正區思源街16號自來水博物館正對面，禮客星巴克隔壁1樓,0223688756,"[星期一: 11:00 – 20:00, 星期二: 11:00 – 20:00, 星期三: ...",4.4,1127,"{'lat': 25.0138677, 'lng': 121.5304218}",https://maps.google.com/?cid=11400017811099986002,2025-05-20


In [58]:
place_id = "ChIJhQuh3l2pQjQRUqi9n8z_NJ4"
gmaps = googlemaps.Client(key=api_key)
detail = gmaps.place(place_id=place_id, language='zh-TW')

In [61]:
detail["result"].keys()

dict_keys(['address_components', 'adr_address', 'business_status', 'curbside_pickup', 'current_opening_hours', 'delivery', 'dine_in', 'formatted_address', 'formatted_phone_number', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'international_phone_number', 'name', 'opening_hours', 'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference', 'reservable', 'reviews', 'serves_beer', 'serves_breakfast', 'serves_brunch', 'serves_dinner', 'serves_lunch', 'serves_vegetarian_food', 'serves_wine', 'takeout', 'types', 'url', 'user_ratings_total', 'utc_offset', 'vicinity', 'website', 'wheelchair_accessible_entrance'])

In [63]:
detail["result"]["adr_address"]

'<span class="postal-code">100</span><span class="country-name">台灣</span><span class="region">台北市</span><span class="locality">中正區</span><span class="street-address">富水里思源街16號</span>自來水博物館正對面，禮客星巴克隔壁1樓'